In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!ls
!unzip ngrok-stable-linux-amd64.zip

--2020-12-10 23:37:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.187.253, 52.2.56.23, 35.174.46.144, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.187.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  19.0MB/s    in 0.7s    

2020-12-10 23:37:30 (19.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

drv  ngrok-stable-linux-amd64.zip
Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!rm -rf /content/*

^C


In [ ]:
!mkdir /content/drv
from google.colab import drive
drive.mount('/content/drv')

mkdir: cannot create directory ‘/content/drv’: File exists
Drive already mounted at /content/drv; to attempt to forcibly remount, call drive.mount("/content/drv", force_remount=True).


In [ ]:
SOURCE = "/content/drv/My Drive/DOAN_201/DAMH"

TRAIN_DIR   = SOURCE + "/Dataset/PlantVillage" + "/train"
VALID_DIR   = SOURCE + "/Dataset/PlantVillage" + "/valid"

MODEL_DIR   = SOURCE + "/models"
LOG_DIR     = SOURCE + "/logs"

In [ ]:
%load_ext tensorboard
from datetime import datetime
from packaging import version

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import json
import datetime
import os

LEARNING_RATE = 0.0001
batch_size = 64
img_height = 299
img_width = 299

IMAGE_SIZE  = (img_height, img_width)

with open(SOURCE+ '/Dataset/PlantVillage/categories.json', 'r') as f:
    cat_to_name = json.load(f)
    classes = list(cat_to_name.values())

print(classes)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Khong___phai_la', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy']


In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=45,#0
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.15, #0
    height_shift_range=0.15, #0
    shear_range=0.1,
    zoom_range=0.5, #0
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",  
    target_size=IMAGE_SIZE,
    batch_size=batch_size)

# valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rescale = 1./255,
#     rotation_range=45,#0
#     horizontal_flip=True,
#     vertical_flip=True,
#     width_shift_range=0.15, #0
#     height_shift_range=0.15, #0
#     shear_range=0.1,
#     zoom_range=0.5, #0
#     fill_mode='nearest')
##################################################




# valid_generator = valid_datagen.flow_from_directory(
#     VALID_DIR, 
#     shuffle=True, 
#     seed=42,
#     color_mode="rgb", 
#     class_mode="categorical",
#     target_size=IMAGE_SIZE,
#     batch_size=batch_size)

Found 19744 images belonging to 25 classes.


In [ ]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150
model = tf.keras.Sequential([
    tf.keras.applications.InceptionV3(input_shape=(img_height,img_width)+(3,),include_top=False, weights='imagenet'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])


latest = tf.train.latest_checkpoint(MODEL_DIR)
if latest:
    print("*"*50)
    print("Load weight from last")
    print(latest)
    model.load_weights(latest)
    print("*"*50)

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

model.summary()

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               67109376  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 25)                12825     
Total params: 88,924,985
Trainable params: 88,890,553
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
now = datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%S")

if not os.path.isdir(LOG_DIR):
    os.mkdir(LOG_DIR)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1, update_freq='batch',profile_batch=0)

if not os.path.isdir(MODEL_DIR):
    os.mkdir(MODEL_DIR)

checkpoint_path = MODEL_DIR + "/cp-" + now + "-{epoch:04d}.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    save_freq='epoch')

#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
EPOCHS=3 #@param {type:"integer"}
STEPS_EPOCHS = train_generator.samples//train_generator.batch_size

In [ ]:
history = model.fit_generator( 
          train_generator,
          steps_per_epoch=STEPS_EPOCHS,
          epochs=EPOCHS,
          # validation_data = valid_generator,
          callbacks=[tensorboard_callback,checkpoint_callback]
          )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
308/308 [==============================] - ETA: 0s - loss: 0.7786 - accuracy: 0.8247 
Epoch 00001: saving model to /content/drv/My Drive/DOAN_201/DAMH/models/cp-2020-12-10--23-38-39-0001.ckpt
308/308 [==============================] - 15455s 50s/step - loss: 0.7786 - accuracy: 0.8247
Epoch 2/3
308/308 [==============================] - ETA: 0s - loss: 0.1100 - accuracy: 0.9693 
Epoch 00002: saving model to /content/drv/My Drive/DOAN_201/DAMH/models/cp-2020-12-10--23-38-39-0002.ckpt
308/308 [==============================] - 15423s 50s/step - loss: 0.1100 - accuracy: 0.9693
Epoch 3/3
215/308 [===================>..........] - ETA: 1:17:34 - loss: 0.0800 - accuracy: 0.9773

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2, 
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    fill_mode='nearest')

test_generator = test_datagen.flow_from_directory(
    VALID_DIR, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
results = model.evaluate_generator(test_generator)
print('test loss, test acc:', results)